### JUnit
JUnit is the defacto Java unit testing framework. The latest version is 5, however here JUnit 4 is discussed as it is more widely used.  Before actually using JUnit, let us write our own testing logic. We'll be testing the below Calculator class:

In [ ]:
class Calculator {
    public int add(int x, int y){
        return x + y;
    }
    
    public int subtract(int x, int y){
        return x - y;
    }
}

To test the functionality, we can write the following:

In [ ]:
class CalculatorTest {
    public static void main(String... args){
        Calculator calc = new Calculator();
        
        int result = calc.add(5, 6);
        int expected = 11;
        
        if(result != expected){
            System.out.println("Unexpected result " + result);
        }
    }
}

It would be better if we use exception to denote test failure. Meanwhile it would be better if we modularise our code, such that testing other aspects of calculator becomes a bit easier.

In [ ]:
class CalculatorTest {
    public int errorCount = 0;
    
    public void testAdd(){
        Calculator calc = new Calculator();
        
        int result = calc.add(5, 6);
        int expected = 11;
        
        if(result != expected){
            throw new IllegalStateException("Unexpected result " + result);
        }
    }
    
    public static void main(String... args){
        CalculatorTest cTest = new CalculatorTest();
        
        try {
            cTest.testAdd();
        } catch (Exception e){
            cTest.errorCount++;
            e.printStackTrace();
        }
        
        if(cTest.errorCount > 0){
            throw new IllegalStateException("There were " + cTest.errorCount + " errors");
        }
    }
}

Using a testing framework like JUnit removes some of the extra code that we have to write. We would just need to define test case, it is the responsibility of the framework to execute it and present the result. Given that, we need to familiarize with some of the associated terms:
- **Test:** a method annotated with `@Test`
- **Test class:** contains one or many tests
- **Suite:** allows us to group test classes
- **Runner:** class runs test.

The equivalent JUnit test class is:

In [ ]:
import static org.junit.Assert.assertEquals;

import org.junit.Test;

public class CalculatorTest {
    
    @Test
    public void testAdd() {
        Calculator calculator = new Calculator();
        int result = calculator.add(5, 6);
        int expected = 11;

        assertEquals("Unexpected result", result, expected);
    }
}

There are a number of assertions available:

In [ ]:
@Test
public void testAssertions() {
    // Uses equals
    assertEquals("Hello", new String("Hello")); // Pass
    assertEquals("Hello", "Hello");             // Pass

    // Uses ==
    assertSame("Hello", new String("Hello"));   // Fail
    assertSame("Hello", "Hello");               // Pass

    // Tests for content of arrays
    assertArrayEquals(new char[] { 'A', 'B', 'C' }, 
                      "ABC".toCharArray());     // Pass
    assertArrayEquals(new String[] { new String("H"), "E", "O" }, 
                      new String[] { "H", "E", "O" }); // Pass
    
    // True false
    assertTrue(5 == 5);    // Pass
    assertFalse(6 > 5);    // Pass

    assertNull(null);      // Pass
}

There are also a negations of above assertions available such as `assertNotNull` and `assertNotSame`  

If some code needs to be repeated before and after every test execution, we can make use of `@Before` and `@After` annotations:

In [ ]:
import org.junit.After;
import org.junit.Before;
import org.junit.Test;

public class CalculatorTest {
    private Calculator calculator;

    @Before
    public void initCalculator() {
        calculator = new Calculator();
    }

    @After
    public void destroyCalculator() {
        calculator = null;
    }

    @Test
    public void testAdd() {
        int result = calculator.add(5, 6);
        int expected = 11;

        assertEquals("Unexpected result", result, expected);
    }

    @Test
    public void testSubtract() {
        int result = calculator.subtract(5, 6);
        int expected = -1;

        assertEquals("Unexpected result", result, expected);
    }
}

There are also a `@BeforeClass` and `@AfterClass` annotations (applied to public static method). These methods run before and after all tests have completed.  

To test for occurance of exception:

In [ ]:
@Test(expected = ArithmeticException.class)
public void testDivisionByZero() {
    calculator.divide(5, 0);
}

To test for timeout:

In [ ]:
@Test(timeout = 10)
public void testTimeout() {
    // Long running task
}

To exclude a test from execution:

In [ ]:
@Test
@Ignore
public void someTestToBeSkipped(){
    // Implementation
}

**Runner:** JUnit by default uses `BlockJUnit4ClassRunner` to run all tests. We can use `@RunWith` annotation to use a custom runner. The `Suite` runner helps grouping test classes together and running the test suite: 

In [ ]:
import org.junit.runner.RunWith;
import org.junit.runners.Suite;
import org.junit.runners.Suite.SuiteClasses;

@RunWith(Suite.class)
@SuiteClasses(value = { CalculatorTest.class, ConverterTest.class })
public class MachineTestSuite {

}

There is another runner called as parameterised runner. It helps testing for multiple values in a single test.

In [ ]:
import org.junit.Test;
import org.junit.runner.RunWith;
import org.junit.runners.Parameterized;
import org.junit.runners.Parameterized.Parameter;
import org.junit.runners.Parameterized.Parameters;

@RunWith(Parameterized.class)
public class CalculatorTest {

    @Parameter(1)
    public int operand1;
    @Parameter(2)
    public int operand2;
    @Parameter(0)
    public int expected;

    @Parameters(name="{index}: {1}+{2}={0}")  // Name parameter helps identify the test case
    public static Collection<Integer[]> data() {
        return Arrays.asList(new Integer[][] { 
            { 0, 0, 0 }, { 0, 5, -5 }, { 10, 6, 4 }, { -6, -3, -3 } // All the test cases
        });
    }

    @Test
    public void testAdd() {
        Calculator calculator = new Calculator();
        
        int result = calculator.add(operand1, operand2);
        assertEquals("Unexpected result", result, expected);
    }
}